I am Reworking the CLass Searcher and Updater Entirely and instead of scraping the website like my original logic used to, I am reworking the code to call on the API directly and parse the JSON instead, since that gives more data along with it being easier to use.

Special thanks to Deej Cabral for this as I would not have used the API without him telling me about it and would be unable to make this project better.

### Getting the Data and Converting to Table

In [1]:
subject = "CSE"
num = "310"

class_name = f"{subject} {num}"

In [2]:
import pandas as pd
import json
import requests

headers = {
  'Authorization': 'Bearer null'
}

response = requests.get("https://eadvs-cscc-catalog-api.apps.asu.edu/catalog-microservices/api/v1/search/classes", headers=headers, params={
    "refine": "Y",
    "campusOrOnlineSelection": "A",
    "catalogNbr": num,
    "honors": "F",
    "promod": "F",
    "searchType": "all",
    "subject": subject,
    "term": "2241"
})

In [3]:
# Parse JSON response
data = json.loads(response.text)

# Extract and format data
formatted_data = []

In [4]:
print(response.text)

{"classes":[{"CLAS":{"STRM":"2241","CLASSNBR":"11378","CRSEID":"104190","CRSEOFFERNBR":"1","CLASSSECTION":"1001","SESSIONCODE":"C","CLASSMTGNBR":"1","SUBJECT":"CSE","SUBJECTFORSORTING":"CSE","CATALOGNBR":"310","CATALOGNBRFORSORTING":"310","CAMPUS":"TEMPE","DESCR":"ASU at Tempe","LOCATION":"TEMPE","DESCR3":"Campus: Tempe","TITLE":"Data Structures and Algorithms","TITLEFORSORTING":"Topic Data Structures and Algorithms","COURSETITLELONG":"Data Structures and Algorithms","STARTDATE":"2024-01-08 00:00:00.0","STARTTIME":"9:00 AM","ENDDATE":"2024-04-26 00:00:00.0","ENDTIME":"10:15 AM","MON":"Y","TUES":"N","WED":"Y","THURS":"N","FRI":"N","SAT":"N","SUN":"N","SCHEDULEPRINT":"Y","FACILITYID":"SCOB210","DESCR1":"Schwada Clsrm/Off 210 (Tempe)","INSTRROLE":"PI","ENRLTOT":"118","ENRLCAP":"150","WAITTOT":"0","WAITCAP":"0","RQMNTDESIGNTN":"","DESCR4":"","DESCRFORMAL":"","CLASSNOTESSEQ":"0","DESCRLONG":"","ACADCAREER":"UGRD","COMPONENTPRIMARY":"LEC","DESCR2":"Lecture","UNITSMINIMUM":"3","UNITSMAXIMUM":

In [5]:
for item in data['classes']:
    class_info = item['CLAS']

    row = {
        'Class': class_name,
        'Class Name': class_info.get('TITLE', ''),
        'Class ID': class_info.get('CLASSNBR', ''),
        'Instructor Name': ", ".join(class_info.get('INSTRUCTORSLIST', [])),
        'Location': class_info.get('LOCATION', ''),
        'Occupancy': f"{class_info.get('ENRLTOT', '')} of {class_info.get('ENRLCAP', '')}"
    }
    
    formatted_data.append(row)

In [6]:
# Convert to Pandas DataFrame
df = pd.DataFrame(formatted_data)

csv_file = "classes.csv"
df.to_csv(csv_file, index=False, encoding='utf-8')

In [7]:
df = pd.read_csv("classes.csv")

# df = pd.read_csv("/home/dark/GitHub/ASU-Class-Searcher/classes.csv")

In [8]:
df.head()

,Class,Class Name,Class ID,Instructor Name,Location,Occupancy
0,CSE 310,Data Structures and Algorithms,11378,Xuerong Feng,TEMPE,118 of 150
1,CSE 310,Data Structures and Algorithms,16437,Xuerong Feng,TEMPE,149 of 156
2,CSE 310,Data Structures and Algorithms,18291,Justin Selgrad,TEMPE,156 of 156
3,CSE 310,Data Structures and Algorithms,24030,kevin ding,TEMPE,156 of 156
4,CSE 310,Data Structures and Algorithms,26224,kevin ding,TEMPE,156 of 156


### Making Sense of the Data

In [9]:
df['Available'] = False 

j = -1

for i in df['Occupancy']:

    j += 1
    numbers = i.split(' of ')
    
    if len(numbers) == 2 and numbers[0].isdigit() and numbers[1].isdigit():
        
        if int(numbers[0]) < int(numbers[1]):
            df.at[j, 'Available'] = True


In [10]:
df.head()

,Class,Class Name,Class ID,Instructor Name,Location,Occupancy,Available
0,CSE 310,Data Structures and Algorithms,11378,Xuerong Feng,TEMPE,118 of 150,True
1,CSE 310,Data Structures and Algorithms,16437,Xuerong Feng,TEMPE,149 of 156,True
2,CSE 310,Data Structures and Algorithms,18291,Justin Selgrad,TEMPE,156 of 156,False
3,CSE 310,Data Structures and Algorithms,24030,kevin ding,TEMPE,156 of 156,False
4,CSE 310,Data Structures and Algorithms,26224,kevin ding,TEMPE,156 of 156,False


In [11]:
abc = df[df['Available'] == True]

In [12]:
abc

,Class,Class Name,Class ID,Instructor Name,Location,Occupancy,Available
0,CSE 310,Data Structures and Algorithms,11378,Xuerong Feng,TEMPE,118 of 150,True
1,CSE 310,Data Structures and Algorithms,16437,Xuerong Feng,TEMPE,149 of 156,True
5,CSE 310,Data Structures and Algorithms,32665,Yiran Luo,TEMPE,148 of 150,True
6,CSE 310,Data Structures and Algorithms,35290,Guoliang Xue,ASUONLINE,76 of 100,True
7,CSE 310,Data Structures and Algorithms,36869,Yiran Luo,TEMPE,73 of 156,True
8,CSE 310,Data Structures and Algorithms,16434,Xuerong Feng,TEMPE,48 of 52,True
9,CSE 310,Data Structures and Algorithms,16435,Xuerong Feng,TEMPE,37 of 52,True
10,CSE 310,Data Structures and Algorithms,16436,Xuerong Feng,TEMPE,37 of 46,True
12,CSE 310,Data Structures and Algorithms,18289,Xuerong Feng,TEMPE,49 of 52,True
14,CSE 310,Data Structures and Algorithms,18293,Xuerong Feng,TEMPE,44 of 52,True
